In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import (train_test_split, cross_val_score, 
                                      GridSearchCV, learning_curve, StratifiedKFold)
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier, 
                               AdaBoostClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (classification_report, confusion_matrix, roc_curve, 
                             auc, accuracy_score, precision_recall_curve, f1_score,
                             precision_score, recall_score, roc_auc_score)
from sklearn.decomposition import PCA, FastICA
from sklearn.manifold import TSNE
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import signal, stats
from scipy.stats import skew, kurtosis
import warnings
warnings.filterwarnings('ignore')

# Beautiful pastel color palette
PASTEL_COLORS = ['#FFB3BA', '#BAFFC9', '#BAE1FF', '#FFFFBA', '#FFD9BA', 
                 '#E0BBE4', '#FFDFD3', '#C7CEEA', '#B4F8C8', '#FBE7C6',
                 '#A8E6CF', '#FFD3B6', '#FFAAA5', '#FF8B94', '#FFC3A0']

# Specific colors for mental states
STATE_COLORS = {
    'relaxed': '#BAFFC9',      # Soft green
    'neutral': '#BAE1FF',      # Soft blue
    'concentrating': '#FFB3BA' # Soft pink/red
}

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette(sns.color_palette(PASTEL_COLORS))

In [45]:
# Load the dataset
try:
    # Try the actual filename from your dataset
    df = pd.read_csv('mental-state.csv')
    print("Dataset loaded successfully: mental-state.csv")
except FileNotFoundError:
    try:
        df = pd.read_csv('emotions.csv')
        print("Dataset loaded successfully: emotions.csv")
    except FileNotFoundError:
        try:
            df = pd.read_csv('EEG_data.csv')
            print("Dataset loaded successfully: EEG_data.csv")
        except FileNotFoundError:
            print("\nDATASET NOT FOUND!")

# Display basic dataset information
print(f"\nDATASET OVERVIEW:")
print(f"{'─'*90}")
print(f"Total Samples: {df.shape[0]:,}")
print(f"Total Features: {df.shape[1]}")
print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print(f"\nFirst 5 Rows:")
print(df.head())

print(f"\nColumn Names:")
print(df.columns.tolist())

print(f"\nData Types:")
print(df.dtypes.value_counts())

print(f"\nMissing Values:")
missing = df.isnull().sum()
if missing.sum() == 0:
    print("No missing values - Clean dataset!")
else:
    print(missing[missing > 0])

# Identify label column (usually the last column or named 'label', 'state', etc.)
label_column = None
for col in ['label', 'state', 'emotion', 'mental_state', 'class']:
    if col in df.columns:
        label_column = col
        break

if label_column is None:
    # Assume last column is the label
    label_column = df.columns[-1]

print(f"\nTarget Column: '{label_column}'")
print(f"\nMental State Distribution:")
print(df[label_column].value_counts())
print(f"\nPercentage Distribution:")
print(df[label_column].value_counts(normalize=True) * 100)

# Basic statistics
print(f"\nStatistical Summary:")
print(df.describe())


DATASET NOT FOUND!

DATASET OVERVIEW:
──────────────────────────────────────────────────────────────────────────────────────────
Total Samples: 2,479
Total Features: 989
Memory Usage: 18.71 MB

First 5 Rows:
   lag1_mean_0  lag1_mean_1  lag1_mean_2  lag1_mean_3  lag1_mean_d_h2h1_0  \
0    25.781648    33.836367   -92.769629    19.187957           -1.542262   
1    29.357891    26.792566   417.203910    19.472121          -38.797263   
2    28.451926    31.076434    72.231301    14.245938          -13.225057   
3    21.282184    19.985184    16.220094    39.787312            1.847866   
4    20.431516    28.982168    27.540246    19.960398            2.491458   

   lag1_mean_d_h2h1_1  lag1_mean_d_h2h1_2  lag1_mean_d_h2h1_3  lag1_mean_q1_0  \
0            0.197462         -119.561133            2.032654       21.596272   
1          -16.897194          -29.368531           -9.055370       44.647424   
2           -0.614138          -28.331698           -8.858742       31.450289   
3   

In [46]:
print("ADVANCED FEATURE ENGINEERING & PREPROCESSING")
print("="*90)

# Separate features and labels
# Remove any non-numeric columns except the label
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()

# If label is in numeric columns and we converted it, remove it
if label_column in numeric_columns and df[label_column].dtype == 'object':
    feature_columns = [col for col in df.columns if col != label_column]
else:
    feature_columns = [col for col in numeric_columns if col != label_column]

X = df[feature_columns].values
y = df[label_column].values
feature_names = feature_columns

print(f"\nFeatures Extracted: {len(feature_names)}")
print(f"Feature Matrix Shape: {X.shape}")
print(f"Target Vector Shape: {y.shape}")
print(f"\nSample Feature Names: {feature_names[:10]}")

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print(f"\nENCODED MENTAL STATES:")
print(f"{'─'*90}")

# Check if classes are already numeric or are strings
for i, class_name in enumerate(le.classes_):
    count = np.sum(y_encoded == i)
    percentage = (count / len(y_encoded)) * 100
    
    # Handle both string and numeric class names
    if isinstance(class_name, str):
        display_name = class_name.upper()
    else:
        # Map numeric labels to mental state names
        state_mapping = {0: 'RELAXED', 1: 'NEUTRAL', 2: 'CONCENTRATING'}
        display_name = state_mapping.get(int(class_name), f'STATE_{int(class_name)}')
    
    print(f"   {i}: {display_name:<20} → {count:,} samples ({percentage:.2f}%)")

# Check for class imbalance
class_counts = pd.Series(y_encoded).value_counts()
imbalance_ratio = class_counts.max() / class_counts.min()
print(f"\nClass Imbalance Ratio: {imbalance_ratio:.2f}:1")
if imbalance_ratio > 1.5:
    print("Moderate class imbalance detected - using stratified sampling")
else:
    print("Classes are relatively balanced")

# Advanced Statistical Feature Engineering
print(f"\nCOMPUTING ADVANCED STATISTICAL FEATURES...")

def compute_advanced_features(X_data):
    """Extract advanced statistical features from EEG signals"""
    advanced_features = []
    
    for row in X_data:
        features = [
            np.mean(row),                    # Mean
            np.std(row),                     # Standard deviation
            np.median(row),                  # Median
            np.var(row),                     # Variance
            skew(row),                       # Skewness
            kurtosis(row),                   # Kurtosis
            np.max(row) - np.min(row),      # Range
            np.percentile(row, 75) - np.percentile(row, 25),  # IQR
            np.percentile(row, 90),         # 90th percentile
            np.percentile(row, 10),         # 10th percentile
        ]
        advanced_features.append(features)
    
    return np.array(advanced_features)

# Compute additional features (optional - can be enabled)
# X_advanced = compute_advanced_features(X)
# X = np.hstack([X, X_advanced])
# print(f"Added {X_advanced.shape[1]} advanced statistical features")
# print(f"New feature matrix shape: {X.shape}")

# Feature Scaling - Critical for Neural Networks and SVM
print(f"\n📏 FEATURE SCALING...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print(f"Features standardized (μ=0, σ=1)")
print(f"caled feature range: [{X_scaled.min():.2f}, {X_scaled.max():.2f}]")

# Outlier Detection (optional info)
outliers = np.abs(X_scaled) > 3
outlier_percentage = (outliers.sum() / X_scaled.size) * 100
print(f"Outliers (>3σ): {outlier_percentage:.2f}% of values")

# Train-Test Split with Stratification
print(f"\nSPLITTING DATA...")
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_encoded
)

print(f"Training Set: {X_train.shape[0]:,} samples ({(len(X_train)/len(X))*100:.1f}%)")
print(f"Testing Set:  {X_test.shape[0]:,} samples ({(len(X_test)/len(X))*100:.1f}%)")

# Verify stratification
print(f"\nTraining Set Distribution:")
for i, class_name in enumerate(le.classes_):
    count = np.sum(y_train == i)
    print(f"{class_name}: {count:,} ({(count/len(y_train))*100:.1f}%)")


ADVANCED FEATURE ENGINEERING & PREPROCESSING

Features Extracted: 988
Feature Matrix Shape: (2479, 988)
Target Vector Shape: (2479,)

Sample Feature Names: ['lag1_mean_0', 'lag1_mean_1', 'lag1_mean_2', 'lag1_mean_3', 'lag1_mean_d_h2h1_0', 'lag1_mean_d_h2h1_1', 'lag1_mean_d_h2h1_2', 'lag1_mean_d_h2h1_3', 'lag1_mean_q1_0', 'lag1_mean_q1_1']

ENCODED MENTAL STATES:
──────────────────────────────────────────────────────────────────────────────────────────
   0: RELAXED              → 819 samples (33.04%)
   1: NEUTRAL              → 830 samples (33.48%)
   2: CONCENTRATING        → 830 samples (33.48%)

Class Imbalance Ratio: 1.01:1
Classes are relatively balanced

COMPUTING ADVANCED STATISTICAL FEATURES...

📏 FEATURE SCALING...
Features standardized (μ=0, σ=1)
caled feature range: [-27.17, 49.61]
Outliers (>3σ): 1.77% of values

SPLITTING DATA...
Training Set: 1,983 samples (80.0%)
Testing Set:  496 samples (20.0%)

Training Set Distribution:
0.0: 655 (33.0%)
1.0: 664 (33.5%)
2.0: 664 (33

In [47]:
print("DIMENSIONALITY REDUCTION ANALYSIS")
print("="*90)

# PCA Analysis
print(f"\nPerforming PCA...")
n_components_pca = min(50, X_scaled.shape[1])
pca = PCA(n_components=n_components_pca)
X_pca = pca.fit_transform(X_scaled)

explained_var_cumsum = np.cumsum(pca.explained_variance_ratio_)
print(f"PCA Complete: {n_components_pca} components")
print(f"First 3 PCs explain: {pca.explained_variance_ratio_[:3].sum()*100:.2f}% variance")
print(f"First 10 PCs explain: {pca.explained_variance_ratio_[:10].sum()*100:.2f}% variance")
print(f"First 20 PCs explain: {pca.explained_variance_ratio_[:20].sum()*100:.2f}% variance")

# Find components needed for 95% variance
n_95 = np.argmax(explained_var_cumsum >= 0.95) + 1
print(f"Components for 95% variance: {n_95}")

# t-SNE for 2D visualization
print(f"\nPerforming t-SNE (this may take 1-2 minutes)...")
n_tsne_samples = min(3000, len(X_scaled))
tsne = TSNE(n_components=2, random_state=42, perplexity=30, learning_rate=200, max_iter=1000)
X_tsne = tsne.fit_transform(X_scaled[:n_tsne_samples])
print(f"t-SNE Complete: Processed {n_tsne_samples:,} samples")


DIMENSIONALITY REDUCTION ANALYSIS

Performing PCA...
PCA Complete: 50 components
First 3 PCs explain: 29.34% variance
First 10 PCs explain: 43.01% variance
First 20 PCs explain: 53.11% variance
Components for 95% variance: 1

Performing t-SNE (this may take 1-2 minutes)...
t-SNE Complete: Processed 2,479 samples


In [48]:
print("TRAINING ADVANCED MACHINE LEARNING MODELS")
print("="*90)

# Define comprehensive model suite
models = {
    'Random Forest': RandomForestClassifier(
        n_estimators=300, 
        max_depth=25, 
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    ),
    'Extra Trees': ExtraTreesClassifier(
        n_estimators=300,
        max_depth=25,
        random_state=42,
        n_jobs=-1
    ),
    'AdaBoost': AdaBoostClassifier(
        n_estimators=100,
        learning_rate=1.0,
        random_state=42
    ),
    'SVM (RBF)': SVC(
        kernel='rbf', 
        C=10, 
        gamma='scale', 
        probability=True,
        random_state=42
    ),
    'Neural Network': MLPClassifier(
        hidden_layer_sizes=(256, 128, 64),
        activation='relu',
        max_iter=1000,
        alpha=0.0001,
        random_state=42,
        early_stopping=True,
        validation_fraction=0.1
    ),
    'K-Nearest Neighbors': KNeighborsClassifier(
        n_neighbors=7,
        weights='distance',
        n_jobs=-1
    ),
    'Logistic Regression': LogisticRegression(
        max_iter=1000,
        random_state=42,
        n_jobs=-1
    )
}

results = {}
trained_models = {}

print(f"\nraining {len(models)} models on {X_train.shape[0]:,} samples...\n")

for idx, (name, model) in enumerate(models.items(), 1):
    print(f"[{idx}/{len(models)}] Training {name}...", end=" ")
    
    # Train model
    model.fit(X_train, y_train)
    trained_models[name] = model
    
    # Predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test) if hasattr(model, 'predict_proba') else None
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Cross-validation (on subset for speed)
    cv_subset_size = min(2000, len(X_train))
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(
        model, X_train[:cv_subset_size], y_train[:cv_subset_size], 
        cv=cv, n_jobs=-1, scoring='accuracy'
    )
    
    results[name] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }
    
    print(f"Acc: {accuracy:.4f} | F1: {f1:.4f} | CV: {cv_scores.mean():.4f}")

# Find best model
best_model_name = max(results.keys(), key=lambda k: results[k]['accuracy'])
best_accuracy = results[best_model_name]['accuracy']

print(f"\n{'='*90}")
print(f"BEST MODEL: {best_model_name}")
print(f"Accuracy: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")
print(f"{'='*90}")

TRAINING ADVANCED MACHINE LEARNING MODELS

raining 7 models on 1,983 samples...

[1/7] Training Random Forest... Acc: 0.9677 | F1: 0.9677 | CV: 0.9597
[2/7] Training Extra Trees... Acc: 0.9698 | F1: 0.9697 | CV: 0.9642
[3/7] Training AdaBoost... Acc: 0.8085 | F1: 0.8076 | CV: 0.8361
[4/7] Training SVM (RBF)... Acc: 0.9536 | F1: 0.9532 | CV: 0.9632
[5/7] Training Neural Network... Acc: 0.9395 | F1: 0.9392 | CV: 0.9481
[6/7] Training K-Nearest Neighbors... Acc: 0.8972 | F1: 0.8952 | CV: 0.8901
[7/7] Training Logistic Regression... Acc: 0.9315 | F1: 0.9312 | CV: 0.9415

BEST MODEL: Extra Trees
Accuracy: 0.9698 (96.98%)


In [ ]:
print("GENERATING 15+ INTERACTIVE VISUALIZATIONS")
print("="*90)

# Ensure class labels are treated as strings for Plotly color mapping
state_color_map = {
    str(le.classes_[i]): STATE_COLORS.get(str(le.classes_[i]), PASTEL_COLORS[i]) 
    for i in range(len(le.classes_))
}

# Helper function to convert numeric labels to string for coloring
def label_to_str(labels):
    return [str(le.classes_[i]) for i in labels]

# 1. 3D PCA Visualization
print("\n[1/15] Creating 3D PCA visualization...")
sample_size = min(3000, len(X_pca))
fig1 = px.scatter_3d(
    x=X_pca[:sample_size, 0], 
    y=X_pca[:sample_size, 1], 
    z=X_pca[:sample_size, 2],
    color=label_to_str(y_encoded[:sample_size]),
    title='<b>3D PCA Visualization - EEG Mental States</b><br><sub>First 3 Principal Components</sub>',
    labels={'x': f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)', 
            'y': f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)', 
            'z': f'PC3 ({pca.explained_variance_ratio_[2]*100:.1f}%)',
            'color': 'Mental State'},
    color_discrete_map=state_color_map,
    opacity=0.6
)
fig1.update_traces(marker=dict(size=3, line=dict(width=0)))
fig1.update_layout(template='plotly_white', font=dict(size=12, family='Arial'), height=650,
                   scene=dict(
                       xaxis=dict(backgroundcolor='rgba(240,240,240,0.5)'),
                       yaxis=dict(backgroundcolor='rgba(240,240,240,0.5)'),
                       zaxis=dict(backgroundcolor='rgba(240,240,240,0.5)')
                   ),
                   legend=dict(title_font_size=12, font_size=11))
fig1.show()

# 2. t-SNE 2D Projection
print("[2/15] Creating t-SNE 2D projection...")
fig2 = px.scatter(
    x=X_tsne[:, 0], 
    y=X_tsne[:, 1],
    color=label_to_str(y_encoded[:n_tsne_samples]),
    title='<b>t-SNE 2D Projection</b><br><sub>Non-linear dimensionality reduction of EEG features</sub>',
    labels={'x': 't-SNE Component 1', 'y': 't-SNE Component 2', 'color': 'Mental State'},
    color_discrete_map=state_color_map
)
fig2.update_traces(marker=dict(size=7, opacity=0.6, line=dict(width=0.5, color='white')))
fig2.update_layout(template='plotly_white', height=600, font=dict(size=12))
fig2.show()

# 3. Comprehensive Model Performance Dashboard
print("[3/15] Creating model performance dashboard...")
model_names = list(results.keys())
accuracies = [results[m]['accuracy'] for m in model_names]
f1_scores = [results[m]['f1_score'] for m in model_names]
precisions = [results[m]['precision'] for m in model_names]
recalls = [results[m]['recall'] for m in model_names]

fig3 = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Accuracy', 'F1-Score', 'Precision', 'Recall'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'bar'}]]
)

# Accuracy
fig3.add_trace(go.Bar(x=model_names, y=accuracies, 
           marker_color=PASTEL_COLORS[:len(model_names)],
           text=[f'{a:.3f}' for a in accuracies],
           textposition='outside',
           name='Accuracy'), row=1, col=1)

# F1-Score
fig3.add_trace(go.Bar(x=model_names, y=f1_scores, 
           marker_color=PASTEL_COLORS[2:2+len(model_names)],
           text=[f'{f:.3f}' for f in f1_scores],
           textposition='outside',
           name='F1'), row=1, col=2)

# Precision
fig3.add_trace(go.Bar(x=model_names, y=precisions, 
           marker_color=PASTEL_COLORS[4:4+len(model_names)],
           text=[f'{p:.3f}' for p in precisions],
           textposition='outside',
           name='Precision'), row=2, col=1)

# Recall
fig3.add_trace(go.Bar(x=model_names, y=recalls, 
           marker_color=PASTEL_COLORS[6:6+len(model_names)],
           text=[f'{r:.3f}' for r in recalls],
           textposition='outside',
           name='Recall'), row=2, col=2)

for i in range(1, 3):
    for j in range(1, 3):
        fig3.update_yaxes(title_text="Score", range=[0, 1.15], row=i, col=j)
        fig3.update_xaxes(tickangle=-45, row=i, col=j)

fig3.update_layout(
    title_text="<b>Comprehensive Model Performance Metrics</b>",
    showlegend=False,
    template='plotly_white',
    height=700,
    font=dict(size=10)
)
fig3.show()

# 4. Confusion Matrix Heatmap
print("[4/15] Creating confusion matrix...")
y_pred_best = results[best_model_name]['y_pred']
cm = confusion_matrix(y_test, y_pred_best)
cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

fig4 = px.imshow(
    cm_percent,
    labels=dict(x="Predicted State", y="True State", color="Accuracy (%)"),
    x=[str(c) for c in le.classes_],
    y=[str(c) for c in le.classes_],
    title=f'<b>Confusion Matrix - {best_model_name}</b><br><sub>Best performing model with {best_accuracy*100:.2f}% accuracy</sub>',
    color_continuous_scale='Blues',
    text_auto=True
)
fig4.update_layout(template='plotly_white', height=550, font=dict(size=12))
fig4.show()

# 5. ROC Curves - Multi-class
print("[5/15] Creating ROC curves...")
if results[best_model_name]['y_pred_proba'] is not None:
    n_classes = len(le.classes_)
    y_test_bin = np.eye(n_classes)[y_test]
    y_pred_proba = results[best_model_name]['y_pred_proba']
    
    fig5 = go.Figure()
    for i, class_name in enumerate(le.classes_):
        fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_pred_proba[:, i])
        roc_auc = auc(fpr, tpr)
        fig5.add_trace(go.Scatter(
            x=fpr, y=tpr,
            mode='lines',
            name=f'{class_name} (AUC={roc_auc:.3f})',
            line=dict(color=state_color_map[str(class_name)], width=4)
        ))
    
    fig5.add_trace(go.Scatter(x=[0,1], y=[0,1], mode='lines', name='Random', line=dict(color='gray', dash='dash', width=2)))
    fig5.update_layout(title='<b>ROC Curves - Multi-class</b>', xaxis_title='False Positive Rate',
                       yaxis_title='True Positive Rate', template='plotly_white', height=600)
    fig5.show()

# 6. Feature Importance (Random Forest)
print("[6/15] Creating feature importance plot...")
if 'Random Forest' in trained_models:
    rf_model = trained_models['Random Forest']
    importances = rf_model.feature_importances_
    indices = np.argsort(importances)[::-1][:25]  # Top 25
    top_feature_names = [feature_names[i] if i < len(feature_names) else f'Feature_{i+1}' for i in indices]
    top_importances = importances[indices]

    fig6 = go.Figure(go.Bar(
        y=top_feature_names[::-1],
        x=top_importances[::-1],
        orientation='h',
        marker=dict(color=top_importances[::-1], colorscale=[[0, PASTEL_COLORS[1]], [0.5, PASTEL_COLORS[3]], [1, PASTEL_COLORS[0]]], showscale=True, colorbar=dict(title="Importance")),
        text=[f'{imp:.4f}' for imp in top_importances[::-1]],
        textposition='outside'
    ))
    fig6.update_layout(title='<b>Top 25 Most Important EEG Features</b>', xaxis_title='Importance Score', template='plotly_white', height=800)
    fig6.show()

# 7. Learning Curves
print("[7/15] Creating learning curves...")
train_sizes = np.linspace(0.1, 1.0, 10)
sample_for_lc = min(1500, len(X_train))
train_sizes_abs, train_scores, val_scores = learning_curve(
    trained_models[best_model_name], 
    X_train[:sample_for_lc], 
    y_train[:sample_for_lc],
    cv=5, train_sizes=train_sizes, n_jobs=-1, scoring='accuracy'
)
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

fig7 = go.Figure()
fig7.add_trace(go.Scatter(x=train_sizes_abs, y=train_mean, mode='lines+markers', name='Training Score', line=dict(color=PASTEL_COLORS[0], width=3), error_y=dict(type='data', array=train_std)))
fig7.add_trace(go.Scatter(x=train_sizes_abs, y=val_mean, mode='lines+markers', name='Validation Score', line=dict(color=PASTEL_COLORS[1], width=3), error_y=dict(type='data', array=val_std)))
fig7.update_layout(title=f'<b>Learning Curves - {best_model_name}</b>', xaxis_title='Number of Training Samples', yaxis_title='Accuracy Score', template='plotly_white', height=550)
fig7.show()

# 8. PCA Explained Variance
print("[8/15] Creating PCA explained variance plot...")
n_components_to_plot = min(30, len(pca.explained_variance_ratio_))
fig8 = go.Figure()
fig8.add_trace(go.Scatter(x=list(range(1,n_components_to_plot+1)), y=explained_var_cumsum[:n_components_to_plot], mode='lines+markers', name='Cumulative', line=dict(color=PASTEL_COLORS[2], width=3), fill='tonexty', fillcolor='rgba(186, 225, 255, 0.3)'))
fig8.add_trace(go.Scatter(x=list(range(1,n_components_to_plot+1)), y=pca.explained_variance_ratio_[:n_components_to_plot], mode='lines+markers', name='Individual', line=dict(color=PASTEL_COLORS[0], width=2)))
fig8.add_hline(y=0.95, line_dash="dash", line_color="gray", annotation_text="95% variance", annotation_position="right")
fig8.update_layout(title='<b>PCA Explained Variance Analysis</b>', xaxis_title='Principal Component Number', yaxis_title='Explained Variance Ratio', template='plotly_white', height=550)
fig8.show()

# 9. Class Distribution Bar
print("[9/15] Creating class distribution visualization...")
class_counts = pd.Series(y).value_counts()
fig9 = go.Figure(data=[go.Bar(x=[str(c) for c in class_counts.index], y=class_counts.values, marker=dict(color=[state_color_map[str(s)] for s in class_counts.index]), text=class_counts.values, textposition='outside')])
fig9.update_layout(title='<b>Mental State Distribution in Dataset</b>', xaxis_title='Mental State', yaxis_title='Number of Samples', template='plotly_white', height=500)
fig9.show()

# 10. Model Comparison Radar Chart
print("[10/15] Creating model comparison radar chart...")
metrics = ['Accuracy', 'F1-Score', 'Precision', 'Recall', 'CV Score']
fig10 = go.Figure()
for idx, model_name in enumerate(model_names):
    model_data = results[model_name]
    values = [model_data['accuracy'], model_data['f1_score'], model_data['precision'], model_data['recall'], model_data['cv_mean']]
    fig10.add_trace(go.Scatterpolar(r=values, theta=metrics, fill='toself', name=model_name, line=dict(color=PASTEL_COLORS[idx], width=2)))
fig10.update_layout(polar=dict(radialaxis=dict(visible=True, range=[0,1])), title='<b>Multi-Metric Model Performance Comparison</b>', template='plotly_white', height=600)
fig10.show()

# 11. Precision-Recall Curves
print("[11/15] Creating precision-recall curves...")
if results[best_model_name]['y_pred_proba'] is not None:
    n_classes = len(le.classes_)
    y_test_bin = np.eye(n_classes)[y_test]
    y_pred_proba = results[best_model_name]['y_pred_proba']
    fig11 = go.Figure()
    for i, class_name in enumerate(le.classes_):
        precision, recall, _ = precision_recall_curve(y_test_bin[:, i], y_pred_proba[:, i])
        fig11.add_trace(go.Scatter(x=recall, y=precision, mode='lines', name=str(class_name), line=dict(color=state_color_map[str(class_name)], width=4)))
    fig11.update_layout(title='<b>Precision-Recall Curves</b>', xaxis_title='Recall', yaxis_title='Precision', template='plotly_white', height=550)
    fig11.show()

# 12. Training Time Comparison
print("[12/15] Creating training time comparison...")
import time
training_times = {}
for name, model in models.items():
    subset_size = min(1000, len(X_train))
    start = time.time()
    model.fit(X_train[:subset_size], y_train[:subset_size])
    training_times[name] = time.time() - start
fig12 = go.Figure(go.Bar(x=list(training_times.keys()), y=list(training_times.values()), text=[f'{t:.2f}s' for t in training_times.values()], textposition='outside'))
fig12.update_layout(title='<b>Model Training Time Comparison</b>', xaxis_title='Model', yaxis_title='Training Time (seconds)', template='plotly_white', height=500)
fig12.show()

# 13. Prediction Confidence Distribution
print("[13/15] Creating prediction confidence distribution...")
if results[best_model_name]['y_pred_proba'] is not None:
    y_pred_proba = results[best_model_name]['y_pred_proba']
    max_confidences = np.max(y_pred_proba, axis=1)
    fig13 = go.Figure()
    for i, class_name in enumerate(le.classes_):
        class_mask = y_test == i
        fig13.add_trace(go.Histogram(x=max_confidences[class_mask], name=str(class_name), marker_color=state_color_map[str(class_name)], opacity=0.7))
    fig13.update_layout(title=f'<b>Prediction Confidence Distribution - {best_model_name}</b>', xaxis_title='Prediction Confidence', yaxis_title='Frequency', template='plotly_white', height=500, barmode='overlay')
    fig13.show()

# 14. Feature Correlation Heatmap
print("[14/15] Creating feature correlation heatmap...")
if X.shape[1] <= 100:
    sample_size = min(1000, X_scaled.shape[0])
    corr_matrix = pd.DataFrame(X_scaled[:sample_size]).corr()
    if corr_matrix.shape[0] <= 50:
        fig14 = px.imshow(corr_matrix, title='<b>EEG Feature Correlation Matrix</b>', color_continuous_scale='RdBu_r', zmin=-1, zmax=1)
        fig14.update_layout(template='plotly_white', height=800)
        fig14.show()
    else:
        print("   ⏭️ Skipping correlation heatmap (too many features)")
else:
    print("   ⏭️ Skipping correlation heatmap (too many features)")

# 15. Cross-Validation Score Distribution
print("[15/15] Creating cross-validation score distribution...")
cv_results = {name: results[name]['cv_mean'] for name in model_names}
cv_stds = {name: results[name]['cv_std'] for name in model_names}
fig15 = go.Figure()
for idx, (name, cv_mean) in enumerate(cv_results.items()):
    fig15.add_trace(go.Box(y=[cv_mean - cv_stds[name]*2, cv_mean - cv_stds[name], cv_mean, cv_mean + cv_stds[name], cv_mean + cv_stds[name]*2], name=name, marker_color=PASTEL_COLORS[idx], boxmean='sd'))
fig15.update_layout(title='<b>Cross-Validation Score Distribution</b>', yaxis_title='Cross-Validation Accuracy', xaxis_title='Model', template='plotly_white', height=550)
fig15.update_xaxes(tickangle=-45)
fig15.show()


GENERATING 15+ INTERACTIVE VISUALIZATIONS

[1/15] Creating 3D PCA visualization...


[2/15] Creating t-SNE 2D projection...


[3/15] Creating model performance dashboard...


[4/15] Creating confusion matrix...


[5/15] Creating ROC curves...


...

[10/15] Creating model comparison radar chart...


[11/15] Creating precision-recall curves...


[12/15] Creating training time comparison...


[13/15] Creating prediction confidence distribution...


[14/15] Creating feature correlation heatmap...
   ⏭️ Skipping correlation heatmap (too many features)
[15/15] Creating cross-validation score distribution...


In [53]:
print("COMPREHENSIVE RESULTS SUMMARY")
print("="*90)

# Create results table
print(f"\n{'Model':<25} {'Accuracy':<10} {'F1-Score':<10} {'Precision':<10} {'Recall':<10} {'CV Score':<12}")
print("─"*90)

sorted_models = sorted(model_names, key=lambda x: results[x]['accuracy'], reverse=True)

for name in sorted_models:
    acc = results[name]['accuracy']
    f1 = results[name]['f1_score']
    prec = results[name]['precision']
    rec = results[name]['recall']
    cv = results[name]['cv_mean']
    cv_std = results[name]['cv_std']
    
    marker = "🏆" if name == best_model_name else "  "
    print(f"{marker} {name:<23} {acc:.4f}     {f1:.4f}     {prec:.4f}     {rec:.4f}     {cv:.4f}±{cv_std:.4f}")

print("\n" + "="*90)
print(f"CHAMPION MODEL: {best_model_name}")
print("="*90)
print(f"Test Accuracy:  {results[best_model_name]['accuracy']:.4f} ({results[best_model_name]['accuracy']*100:.2f}%)")
print(f"F1-Score:       {results[best_model_name]['f1_score']:.4f}")
print(f"Precision:      {results[best_model_name]['precision']:.4f}")
print(f"Recall:         {results[best_model_name]['recall']:.4f}")
print(f"CV Accuracy:    {results[best_model_name]['cv_mean']:.4f} ± {results[best_model_name]['cv_std']:.4f}")

# Detailed classification report
print("\n" + "="*90)
print(f"DETAILED CLASSIFICATION REPORT - {best_model_name}")
print("="*90)

# Convert target names to strings to avoid float issues
target_names_str = [str(c) for c in le.classes_]
print(classification_report(y_test, y_pred_best, target_names=target_names_str, digits=4))

# Per-class performance
print("\n" + "="*90)
print("PER-CLASS PERFORMANCE BREAKDOWN")
print("="*90)

for i, class_name in enumerate(le.classes_):
    class_mask = y_test == i
    class_predictions = y_pred_best[class_mask]
    class_accuracy = accuracy_score(y_test[class_mask], class_predictions)
    
    print(f"\n{str(class_name).upper()}:")
    print(f"Samples in test set: {np.sum(class_mask)}")
    print(f"Correctly predicted: {np.sum(class_predictions == i)}")
    print(f"Class accuracy:      {class_accuracy:.4f} ({class_accuracy*100:.2f}%)")


COMPREHENSIVE RESULTS SUMMARY

Model                     Accuracy   F1-Score   Precision  Recall     CV Score    
──────────────────────────────────────────────────────────────────────────────────────────
🏆 Extra Trees             0.9698     0.9697     0.9702     0.9698     0.9642±0.0101
   Random Forest           0.9677     0.9677     0.9687     0.9677     0.9597±0.0106
   SVM (RBF)               0.9536     0.9532     0.9548     0.9536     0.9632±0.0108
   Neural Network          0.9395     0.9392     0.9395     0.9395     0.9481±0.0093
   Logistic Regression     0.9315     0.9312     0.9312     0.9315     0.9415±0.0030
   K-Nearest Neighbors     0.8972     0.8952     0.9037     0.8972     0.8901±0.0094
   AdaBoost                0.8085     0.8076     0.8315     0.8085     0.8361±0.0138

CHAMPION MODEL: Extra Trees
Test Accuracy:  0.9698 (96.98%)
F1-Score:       0.9697
Precision:      0.9702
Recall:         0.9698
CV Accuracy:    0.9642 ± 0.0101

DETAILED CLASSIFICATION REPORT - Extra

In [56]:
print("KEY INSIGHTS & STATISTICAL ANALYSIS")
print("="*90)

# Convert labels to strings to avoid float issues
class_names_str = [str(c) for c in le.classes_]

print(f"\nDATASET STATISTICS:")
print(f"Total Samples:        {len(df):,}")
print(f"EEG Features:         {len(feature_names)}")
print(f"Mental States:        {len(class_names_str)} ({', '.join(class_names_str)})")
print(f"Train/Test Split:     {len(X_train):,} / {len(X_test):,} ({(len(X_train)/(len(X_train)+len(X_test)))*100:.0f}% / {(len(X_test)/(len(X_train)+len(X_test)))*100:.0f}%)")
print(f"Class Balance Ratio:  {imbalance_ratio:.2f}:1")

print(f"\nMODEL PERFORMANCE:")
print(f"Best Accuracy:        {max(r['accuracy'] for r in results.values()):.4f} ({best_model_name})")
print(f"Average Accuracy:     {np.mean([r['accuracy'] for r in results.values()]):.4f}")
print(f"Worst Accuracy:       {min(r['accuracy'] for r in results.values()):.4f}")
print(f"Performance Range:    {max(r['accuracy'] for r in results.values()) - min(r['accuracy'] for r in results.values()):.4f}")
print(f"Models > 90% Acc:     {sum(1 for r in results.values() if r['accuracy'] > 0.90)}/{len(models)}")

print(f"\nDIMENSIONALITY REDUCTION:")
print(f"Original Features:    {X.shape[1]}")
print(f"PCA (3 components):   {pca.explained_variance_ratio_[:3].sum()*100:.2f}% variance retained")
print(f"PCA (10 components):  {pca.explained_variance_ratio_[:10].sum()*100:.2f}% variance retained")
print(f"PCA (20 components):  {pca.explained_variance_ratio_[:20].sum()*100:.2f}% variance retained")
print(f"Components for 95%:   {n_95} components")

if 'Random Forest' in trained_models:
    rf = trained_models['Random Forest']
    print(f"\nRANDOM FOREST INSIGHTS:")
    print(f"Number of Trees:      {rf.n_estimators}")
    print(f"Max Tree Depth:       {rf.max_depth}")
    print(f"Top Feature Import:   {rf.feature_importances_.max():.4f}")
    print(f"Min Feature Import:   {rf.feature_importances_.min():.6f}")
    print(f"Features > 1% import: {np.sum(rf.feature_importances_ > 0.01)}")

if 'Neural Network' in trained_models:
    nn = trained_models['Neural Network']
    print(f"\nNEURAL NETWORK INSIGHTS:")
    print(f"Architecture:         {nn.hidden_layer_sizes}")
    print(f"Total Parameters:     ~{sum((nn.coefs_[i].shape[0] * nn.coefs_[i].shape[1]) for i in range(len(nn.coefs_))):,}")
    print(f"Iterations:           {nn.n_iter_}")
    print(f"Final Loss:           {nn.loss_:.6f}")

# Confusion analysis
print(f"\nCONFUSION ANALYSIS:")
cm = confusion_matrix(y_test, y_pred_best)
total_errors = len(y_test) - np.trace(cm)
error_rate = total_errors / len(y_test)
print(f"Total Misclassified:  {total_errors} / {len(y_test)} ({error_rate*100:.2f}%)")

for i, class_name in enumerate(class_names_str):
    for j, other_class in enumerate(class_names_str):
        if i != j and cm[i, j] > 0:
            print(f"   {class_name} → {other_class}:  {cm[i, j]} samples ({cm[i, j]/cm[i].sum()*100:.1f}%)")


KEY INSIGHTS & STATISTICAL ANALYSIS

DATASET STATISTICS:
Total Samples:        2,479
EEG Features:         988
Mental States:        3 (0.0, 1.0, 2.0)
Train/Test Split:     1,983 / 496 (80% / 20%)
Class Balance Ratio:  1.01:1

MODEL PERFORMANCE:
Best Accuracy:        0.9698 (Extra Trees)
Average Accuracy:     0.9240
Worst Accuracy:       0.8085
Performance Range:    0.1613
Models > 90% Acc:     5/7

DIMENSIONALITY REDUCTION:
Original Features:    988
PCA (3 components):   29.34% variance retained
PCA (10 components):  43.01% variance retained
PCA (20 components):  53.11% variance retained
Components for 95%:   1 components

RANDOM FOREST INSIGHTS:
Number of Trees:      300
Max Tree Depth:       25
Top Feature Import:   0.0199
Min Feature Import:   0.000008
Features > 1% import: 11

NEURAL NETWORK INSIGHTS:
Architecture:         (256, 128, 64)
Total Parameters:     ~294,080
Iterations:           19
Final Loss:           0.000648

CONFUSION ANALYSIS:
Total Misclassified:  15 / 496 (3.02%

In [57]:
# ======= DASHBOARD EXPORTER CELL =======

import os, pickle, json
import numpy as np
import pandas as pd

# Helper to save with joblib/pickle
try:
    import joblib
except:
    joblib = None

# Collect objects from notebook globals
g = globals()

# Prefer these names (common in your notebook)
candidates = {
    'df': g.get('df', None),
    'feature_names': g.get('feature_names', None),
    'X_scaled': g.get('X_scaled', None),
    'X_pca': g.get('X_pca', None),
    'X_tsne': g.get('X_tsne', None),
    'y_encoded': g.get('y_encoded', None),
    'y': g.get('y', None),
    'y_train': g.get('y_train', None),
    'y_test': g.get('y_test', None),
    'le': g.get('le', None),
    'trained_models': g.get('trained_models', None),
    'results': g.get('results', None)
}

# Import sklearn tools for fallback computations
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, LabelEncoder

# If we only have df or raw columns, try to derive X and y
if candidates['X_scaled'] is None:
    # Try to build X_scaled from df and feature_names
    if candidates['df'] is not None:
        df = candidates['df']
        if candidates['feature_names'] is None:
            # choose numeric columns except last if label at last
            numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
            label_candidates = ['label','state','emotion','mental_state','class','target']
            label_col = None
            for c in label_candidates:
                if c in df.columns:
                    label_col = c
                    break
            if label_col is None:
                # assume last column is label if non-numeric
                if df.columns[-1] not in numeric_cols:
                    label_col = df.columns[-1]
            if label_col:
                features = [c for c in df.columns if c != label_col and c in numeric_cols]
            else:
                features = numeric_cols
            candidates['feature_names'] = features
        try:
            X = df[candidates['feature_names']].values
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)
            candidates['X_scaled'] = X_scaled
        except Exception as e:
            pass

# Ensure y or y_encoded exists
if candidates['y_encoded'] is None and candidates['y'] is not None:
    # encode
    if isinstance(candidates['y'], (pd.Series, np.ndarray, list)):
        le = LabelEncoder().fit(candidates['y'])
        candidates['y_encoded'] = le.transform(candidates['y'])
        candidates['le'] = le

if candidates['y'] is None and candidates['y_encoded'] is not None and candidates['le'] is not None:
    try:
        candidates['y'] = candidates['le'].inverse_transform(candidates['y_encoded'])
    except:
        pass

# Compute PCA if missing
if candidates['X_pca'] is None and candidates['X_scaled'] is not None:
    try:
        n = min(3, candidates['X_scaled'].shape[1])
        pca = PCA(n_components=n, random_state=42)
        candidates['X_pca'] = pca.fit_transform(candidates['X_scaled'])
    except Exception as e:
        candidates['X_pca'] = None

# Compute t-SNE if missing
if candidates['X_tsne'] is None and candidates['X_scaled'] is not None:
    try:
        n_samples = min(2000, candidates['X_scaled'].shape[0])
        tsne = TSNE(n_components=2, random_state=42, learning_rate=200, perplexity=30, init='pca', metric='euclidean')
        candidates['X_tsne'] = tsne.fit_transform(candidates['X_scaled'][:n_samples])
    except Exception as e:
        candidates['X_tsne'] = None

# For results: if results is a dict, convert to DataFrame
if candidates['results'] is not None:
    if isinstance(candidates['results'], dict):
        try:
            results_df = pd.DataFrame.from_dict(candidates['results'], orient='index')
            results_df = results_df.reset_index().rename(columns={'index':'Model'})
            candidates['results'] = results_df
        except:
            pass

# Save numeric arrays into NPZ file
npz_dict = {}
for k in ['X_scaled','X_pca','X_tsne','y_encoded','y_train','y_test']:
    v = candidates.get(k)
    if v is not None:
        try:
            npz_dict[k] = np.array(v)
        except:
            pass

# Save dataframe-ish objects as CSV/pickle
if candidates.get('df') is not None:
    try:
        candidates['df'].to_csv('dashboard_df.csv', index=False)
    except:
        pass

# Save le classes
if candidates.get('le') is not None:
    try:
        with open('label_encoder_classes.json', 'w') as f:
            json.dump(list(candidates['le'].classes_), f)
    except:
        pass

# Save results as CSV/pickle
if candidates.get('results') is not None:
    try:
        candidates['results'].to_csv('dashboard_results.csv', index=False)
    except:
        try:
            with open('results.pkl','wb') as f:
                pickle.dump(candidates['results'], f)
        except:
            pass

# Save feature names
if candidates.get('feature_names') is not None:
    with open('feature_names.json','w') as f:
        json.dump(list(candidates['feature_names']), f)

# Save npz
if len(npz_dict) > 0:
    np.savez_compressed('dashboard_artifacts.npz', **npz_dict)
    print("Saved dashboard_artifacts.npz with arrays:", list(npz_dict.keys()))
else:
    print("No arrays saved to NPZ (X_scaled/X_pca/X_tsne/y_encoded etc missing)")

# Save trained models if present
if candidates.get('trained_models') is not None:
    models = candidates['trained_models']
    # joblib preferred
    try:
        if joblib:
            joblib.dump(models, 'trained_models.joblib')
        else:
            with open('trained_models.pkl', 'wb') as f:
                pickle.dump(models, f)
        print("Saved trained models to trained_models.joblib/pkl")
    except Exception as e:
        print("Failed to save trained models:", e)

print("Export complete. Files created (if data found): dashboard_artifacts.npz, trained_models.joblib/pkl, dashboard_results.csv (or results.pkl), feature_names.json, label_encoder_classes.json, dashboard_df.csv")
# ======= END EXPORTER CELL =======


Saved dashboard_artifacts.npz with arrays: ['X_scaled', 'X_pca', 'X_tsne', 'y_encoded', 'y_train', 'y_test']
Saved trained models to trained_models.joblib/pkl
Export complete. Files created (if data found): dashboard_artifacts.npz, trained_models.joblib/pkl, dashboard_results.csv (or results.pkl), feature_names.json, label_encoder_classes.json, dashboard_df.csv
